In [76]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from tensorflow import keras as k

import tensorflow as tf
import pandas as pd
import numpy as np

# %pip install d2l==1.0.0a1.post0
from d2l import tensorflow as d2l


In [77]:
data = pd.read_csv('./input/dataset.csv')
# dataImputed = pd.read_csv('./input/datasetImputed.csv')

In [78]:
# è stata riscritta una funzione di reshape inutile 
# e.g. (a_prev=x_train, season=trn_ssn)
def reshape_to_inputshape(a_prev,season):
    totalMatches = len(season)*38
    input_step = int(a_prev.shape[0]/totalMatches)
    prev_f = a_prev.shape[1]
    return np.reshape(a_prev, (totalMatches, input_step, prev_f))

In [141]:
def decode(x):
    print(x)
    if x == 0:
        v = [1,0,0]
    elif x == 1:
        v = [0,1,0]
    elif x == 2:
        v = [0,0,1]
    return np.array(v)

In [146]:
def report(model, x_train, y_train, x_test, y_test, encoder):
    #
    y_pred = model.predict(x_test)
    print("shape y_pred", y_pred.shape)
    y_predm = np.asarray(y_pred)
    print("shape y_predm pre", y_predm.shape)
    y_predm = np.argmax(y_predm, axis=2)
    print("shape y_predm post", y_predm.shape)
    y_predm = np.vectorize(decode, signature='(n,m) -> (k,3)')(y_predm)
    print("shape y_predm vect", y_predm.shape)
    
    y_testm = np.argmax(y_test, axis=2)
    y_testm = np.vectorize(decode, signature='(n,m) -> (k,3)')(y_testm)
    print("shape y_testm", y_testm.shape)

    #
    # y_pred_train = model.predict(x_train)
    # print("shape y_pred_train", y_pred_train.shape)
    # y_pred_train = np.asarray(y_pred_train)
    # print("shape y_pred_train", y_pred_train.shape)
    # y_predm_train = np.argmax(y_pred_train, axis=2).ravel()
    # print("shape y_predm_train", y_predm_train.shape)
    # y_trainm = np.argmax(y_train, axis = 2).ravel()
    # print("shape y_trainm", y_trainm.shape)

    # inverse
    y_predm = encoder.inverse_transform(y_predm)
    print("shape y_predm", y_predm.shape)
    y_testm = encoder.inverse_transform(y_testm)
    print("shape y_testm", y_testm.shape)

    # y_predm_train = encoder.inverse_transform(y_predm_train)
    # print("shape y_predm_train", y_predm_train.shape)
    # y_trainm = encoder.inverse_transform(y_trainm)
    # print("shape y_trainm", y_trainm.shape)

    #Model Metrics
    print(classification_report(y_testm, y_predm, digits=3))

# Preprocessamento dati

In [81]:
features = ['HomeTeam', 'AwayTeam', 
            'HTeamEloScore', 'ATeamEloScore', 
            'HTdaysSinceLastMatch', 'ATdaysSinceLastMatch', 
            'HTW_rate', 'ATW_rate', 'ATD_rate', 'HTD_rate', 
            '7_HTW_rate', '12_HTW_rate', '7_ATW_rate', '12_ATW_rate', 
            '7_HTD_rate', '12_HTD_rate', '7_ATD_rate', '12_ATD_rate',
            '7_HTL_rate', '12_HTL_rate', '7_ATL_rate', '12_ATL_rate',
            '5_HTHW_rate', '5_ATAW_rate']

X = pd.get_dummies(data[features])

# Se non cambiamo nulla il OneHotEncoder assegna:
# A -> 1 0 0
# D -> 0 1 0
# H -> 0 0 1
y = data[['FTR']].to_numpy().ravel().reshape(-1, 1)
enc = OneHotEncoder(sparse=False)
y = enc.fit_transform(y)
X_imputed = SimpleImputer().fit_transform(X)


In [82]:
trn_ssn = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
trn_ssn_len = len(trn_ssn)
tst_ssn = [2016,2017,2018] 
tst_ssn_len = len(tst_ssn)

test_size = float(tst_ssn_len)/(tst_ssn_len+trn_ssn_len)

#Split X and Y into training and Test Sets
x_train, x_test, y_train, y_test = train_test_split(X_imputed, y, shuffle=False, test_size=test_size)

# RandomForestClassifier

In [83]:
forest = RandomForestClassifier(n_estimators=2, random_state=2)
forest = forest.fit(x_train, y_train)


#Forest Model Metrics
print("Forest Classifier")
print("Train Score: ", forest.score(x_train, y_train))
print("Test Score: ", forest.score(x_test, y_test))

Forest Classifier
Train Score:  0.6302631578947369
Test Score:  0.22017543859649122


In [84]:
n = 10
m = 5
max_depth = 10
forests = []
grid = [{"n_estimators": list(range(1, n)), "random_state": list(range(0, m)), "max_depth": list(range(1, max_depth))}]
gridSearch = GridSearchCV(RandomForestClassifier(), param_grid=grid, n_jobs=10, return_train_score=True)
gridSearch.fit(x_train, y_train)

print("Forest Classifiers Best Score: ", gridSearch.best_score_)
print("Forest Classifiers Best Params: ", gridSearch.best_params_)
print("Forest Classifiers Best Params: ", gridSearch.best_estimator_)

Forest Classifiers Best Score:  0.4305921052631579
Forest Classifiers Best Params:  {'max_depth': 8, 'n_estimators': 1, 'random_state': 1}
Forest Classifiers Best Params:  RandomForestClassifier(max_depth=8, n_estimators=1, random_state=1)


# Neural Network

In [85]:
y_nn = ((data[['ordinalHR']]).to_numpy()*2)
#Split X and Y into training and Test Sets
# y_nn = OneHotEncoder(sparse=True).fit_transform(y_nn)
x_train_nn, x_test_nn, y_train_nn, y_test_nn = train_test_split(X_imputed, y_nn, shuffle=True)

In [86]:
nn = k.models.Sequential([
    k.layers.Flatten(),
    k.layers.Dense(41, activation='relu'),
    k.layers.Dense(75, activation='relu'),
    k.layers.Dropout(0.3),
    k.layers.Dense(3, activation='softmax'),
])

In [87]:
learning_rate=0.01

nn.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=k.optimizers.Adam(learning_rate),
    metrics=['accuracy']
)
nn(x_train_nn)
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (3135, 94)                0         
                                                                 
 dense_15 (Dense)            (3135, 41)                3895      
                                                                 
 dense_16 (Dense)            (3135, 75)                3150      
                                                                 
 dropout_7 (Dropout)         (3135, 75)                0         
                                                                 
 dense_17 (Dense)            (3135, 3)                 228       
                                                                 
Total params: 7,273
Trainable params: 7,273
Non-trainable params: 0
_________________________________________________________________


In [88]:
epochs=10
batch_size=50
nn.fit(x_train_nn, y_train_nn, epochs=epochs, batch_size=batch_size)

Epoch 1/10
 6/63 [=>............................] - ETA: 0s - loss: 5.4536 - accuracy: 0.4767 

2022-11-13 14:00:35.518665: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 1s 8ms/step - loss: 1.8136 - accuracy: 0.4896
Epoch 2/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9604 - accuracy: 0.5512
Epoch 3/10
63/63 [==============================] - 0s 7ms/step - loss: 0.9492 - accuracy: 0.5569
Epoch 4/10
63/63 [==============================] - 0s 7ms/step - loss: 0.9447 - accuracy: 0.5541
Epoch 5/10
63/63 [==============================] - 0s 7ms/step - loss: 0.9457 - accuracy: 0.5557
Epoch 6/10
63/63 [==============================] - 0s 7ms/step - loss: 0.9418 - accuracy: 0.5550
Epoch 7/10
63/63 [==============================] - 0s 7ms/step - loss: 0.9417 - accuracy: 0.5640
Epoch 8/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9431 - accuracy: 0.5550
Epoch 9/10
63/63 [==============================] - 0s 7ms/step - loss: 0.9423 - accuracy: 0.5541
Epoch 10/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9396 - accuracy: 0.5646


In [89]:
print(nn.evaluate(x_test_nn, y_test_nn))

21/33 [==================>...........] - ETA: 0s - loss: 0.9223 - accuracy: 0.5640

2022-11-13 14:00:40.004502: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 6ms/step - loss: 0.9100 - accuracy: 0.5799
[0.9099838137626648, 0.579904317855835]


# Lstm preprocessing

In [90]:
#Setup XY to have 10 game steps
x_train_lstm = reshape_to_inputshape(x_train,trn_ssn)
y_train_lstm = reshape_to_inputshape(y_train,trn_ssn)
# y_train = np.moveaxis(y_train, 0, 1)

x_test_lstm = reshape_to_inputshape(x_test,tst_ssn)
y_test_lstm = reshape_to_inputshape(y_test,tst_ssn)
# y_test = np.moveaxis(y_test, 0, 1)

Tx = x_train_lstm.shape[1] #Time steps
Ty = y_train_lstm.shape[0] #Time Steps

num_features = x_train_lstm.shape[2] #Features per step
inputs = tf.keras.Input(shape=(Tx, num_features))

# LSTM

In [94]:
model_path = './models/lstm/'
# Check se il modello è già salvato
model = k.models.load_model(model_path)
if model == None:
    model = k.models.Sequential(
        [
            k.layers.LSTM(64, return_sequences=True),
            k.layers.Dropout(0.4),
            k.layers.Dense(1000, activation="relu"),
            k.layers.Dropout(0.3),
            k.layers.Dense(250, activation="relu"),
            k.layers.Dropout(0.2),
            k.layers.Dense(3, activation="softmax")
        ]
    )

    model.compile(
        loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.0001),
        metrics=["accuracy"]
    )

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

    model.fit(x_train_lstm, y_train_lstm, epochs=10, callbacks=[callback])
    model.save('./models/lstm')

In [147]:
model.evaluate(x_test_lstm, y_test_lstm)
report(model, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

4/4 [==============================] - 0s 11ms/step
shape y_pred (114, 10, 3)
shape y_predm pre (114, 10, 3)
shape y_predm post (1140,)
shape y_predm post (1140, 1)
0
0
0
0
2
0
2
0
2
0
2
2
0
0
2
0
0
0
2
2
2
2
0
2
0
2
2
2
2
2
2
2
0
0
2
2
2
1
2
0
0
2
0
2
2
0
2
2
2
0
2
0
2
0
2
2
0
2
1
0
0
2
0
1
0
2
2
0
2
2
2
2
2
0
2
2
2
0
0
2
2
2
2
2
0
0
0
2
2
2
2
2
2
0
0
2
0
0
2
0
2
2
1
2
2
2
0
2
0
2
2
2
1
0
2
2
2
0
0
0
2
2
0
2
2
2
0
2
2
2
0
0
2
2
2
2
0
0
0
0
2
1
2
1
2
1
2
2
2
2
2
0
0
2
2
2
0
0
2
0
0
0
0
0
2
0
2
2
2
0
0
2
2
0
2
2
0
2
2
2
1
0
2
2
2
2
2
2
2
2
0
2
0
0
2
0
0
0
2
2
2
2
2
2
0
0
0
0
0
0
2
0
2
0
2
0
2
2
2
2
2
0
2
2
0
1
0
0
2
0
0
2
2
2
2
2
2
2
0
0
2
2
2
2
0
2
0
2
0
1
0
2
2
2
1
0
2
2
0
2
0
2
2
2
2
2
2
0
0
2
0
2
1
2
0
0
2
0
2
2
2
2
0
2
0
2
2
2
0
2
2
2
2
2
0
2
2
0
2
2
2
2
2
0
2
0
2
1
2
0
0
2
0
0
0
2
0
2
0
2
2
2
0
0
1
0
2
1
0
2
2
2
0
2
2
2
2
2
0
0
2
0
2
0
0
0
0
0
2
2
2
0
2
2
2
0
0
2
2
1
2
0
0
2
0
2
2
2
2
0
0
0
2
2
2
0
2
2
2
2
0
2
0
2
2
2
0
2
2
2
0
0
0
2
2
1
2
2
2
2
2
2
1
2
0
1
0
2
2
2
1
0
2
2
2
0
0
0

ValueError: setting an array element with a sequence.

## 2 LSTM concatenate + dense e dropout in sequenza

In [ ]:
inputs = k.layers.Input(shape=(10,94))
lstm1 = k.layers.LSTM(100, return_sequences=True, activation="relu")(inputs)
lstm2 = k.layers.LSTM(50, return_sequences=True, activation="relu")(inputs)
concateneted = k.layers.Concatenate()([
    lstm1,
    lstm2
])

out = k.layers.Dropout(0.5)(concateneted)
out = k.layers.Dense(1000, activation="relu")(out)
out = k.layers.Dropout(0.5)(out)
out = k.layers.Dense(250, activation="relu")(out)
out = k.layers.Dropout(0.5)(out)
out = k.layers.Dense(3, activation="softmax")(out)

model = k.models.Model(inputs=inputs, outputs=out)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

model.fit(x_train_lstm, y_train_lstm, epochs=500, callbacks=[callback])

In [ ]:
model.evaluate(x_test_lstm, y_test_lstm)
report(model, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

### Fit
loss: 0.9373 - accuracy: 0.5609
### Evaluate
loss: 0.9192 - accuracy: 0.5930
### Classification report

| | precision | recall | f1-score | support |
| - | ----------- | ------ | -------- | ------- |
| A | 0.609 | 0.614 | 0.612 | 345 |
| D | 0.321 | 0.201 | 0.247 | 254 |
| H | 0.652 | 0.763 | 0.704 | 541 |
| accuracy | |  | 0.593 | 1140 |
| macro avg | 0.527 | 0.526 | 0.521 | 1140 |
| weighted avg | 0.565 | 0.593 | 0.574 | 1140 |

## LSTM in parallelo

In [ ]:
inputs = k.layers.Input(shape=(10,94))
x = []
for t in range(Tx):
    module = k.layers.LSTM((t+1)*10, return_sequences=True)(inputs)
    module = k.layers.Dropout(0.7)(module)
    x.append(module)
x = k.layers.Concatenate()(x)

out = k.layers.Dense(1000, activation="relu")(x)
out = k.layers.Dropout(0.7)(out)
out = k.layers.Dense(250, activation="relu")(out)
out = k.layers.Dense(3, activation="softmax")(out)

txLstm = k.models.Model(inputs=inputs, outputs=out)

In [ ]:
txLstm.summary()

In [ ]:
txLstm.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

txLstm.fit(x_train_lstm, y_train_lstm, epochs=500, callbacks=[callback], batch_size=16)

In [ ]:
txLstm.evaluate(x_train_lstm, y_train_lstm)

In [ ]:
txLstm.evaluate(y_test_lstm, y_test_lstm)

# GRU

In [ ]:
gru = k.models.Sequential([
    k.layers.GRU(100, return_sequences=True, activation="relu"),
    k.layers.Dropout(0.4),
    k.layers.Dense(1000, activation="relu"),
    k.layers.Dropout(0.3),
    k.layers.Dense(250, activation="relu"),
    k.layers.Dropout(0.2),
    k.layers.Dense(3, activation="softmax")
])

In [ ]:
gru(x_train_lstm)
gru.summary()

In [ ]:
gru.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

gru.fit(x_train_lstm, y_train_lstm, epochs=500, callbacks=[callback])

In [ ]:
gru.evaluate(x_test_lstm, y_test_lstm)
report(gru, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)